### Assignment 2: word2vec

#### 1.a  Show that: $ - \sum_{w \in \text{Vocab}} y_w \log \hat y_w = -\log ( \hat(y)_o ) $, answer should be in one line.

$\textbf{Answer}$: The ground truth $y_w$ is a one-hot-vector with only the component w.r.t. the outside word being 1, the only term left is $-\log ( \hat(y)_o )$ 

#### 1.b Compute the partial derivative of $J_{naive-softmax}(v_c, o, U)$ w.r.t. $v_c$.
$$
\begin{align}
J_{naive-softmax}(v_c,o,U) &= -\log \hat y_o  \\
\hat y_o &= \frac{\exp(e_o)}{\sum_{w=1}^V \exp(e_w)} \\
e_o &= u_o^Tv_c
\end{align}
$$


Use chain rule of derivatives: $\frac{\partial J}{\partial v_c} = \frac{\partial J}{\partial \hat y_o} \frac{\partial \hat y_o}{\partial e_o} \frac{\partial e_o}{\partial v_c}$ where:

$$
\begin{align}
\frac{\partial J}{\partial \hat y_o} &= -\frac{1}{\hat y_o}  \\
\frac{\partial \hat y_o}{\partial e_o} &=  \frac{\exp(e_o)}{\sum_{w=1}^V \exp(e_w)} - \frac{(\exp(e_o))^2}{ (\sum_{w=1}^V \exp(e_w))^2} = \hat y_o - \hat y_o ^2  \\
\frac{\partial e_o}{\partial v_c} &= u_o
\end{align}
$$

$$
\begin{align}
\frac{\partial J}{\partial v_c} &= (\hat y_o - 1) u_o \\
&= U y \cdot (y^T (\hat y - y))
\end{align}
$$

Where the $\cdot$ is the product between a vector and a scalar.

#### 1.c Compute the partial derivative of $J_{naive-softmax}(v_c, o, U)$ w.r.t. $u_w$

The chain rule of derivatives will have two terms in common compared to 1.b:
$$
\frac{\partial J}{\partial u_o} = \frac{\partial J}{\partial \hat y_o} \frac{\partial \hat y_o}{\partial e_o} \frac{\partial e_o}{\partial u_w}
$$
The first term $\frac{\partial J}{\partial \hat y_o}=-\frac{1}{\hat y_o}$ will be the same, but the 2nd and 3rd terms depend on whether $w=o$ or not.

1. When $w=o$, 
$$
\begin{align}
\frac{\partial \hat y_o}{\partial e_o} &=  \frac{\exp(e_o)}{\sum_{w=1}^V \exp(e_w)} - \frac{(\exp(e_o))^2}{ (\sum_{w=1}^V \exp(e_w))^2} = \hat y_o - \hat y_o ^2  \\
\frac{\partial e_o}{\partial u_w} &= v_c
\end{align}
$$
In this case $\frac{\partial J}{\partial u_w} = (\hat y_w - 1)  v_c$  
  
2. When $w \ne o$,
$$
\begin{align}
\frac{\partial \hat y_o}{\partial e_w} &= - \frac{\exp(e_o) \exp(e_w)}{ (\sum_{w=1}^V \exp(e_w))^2} = -\hat y_w  \hat y_o   \\
\frac{\partial e_w}{\partial u_w} &= v_c
\end{align}
$$
In this case $ \frac{\partial J}{\partial u_w} = \hat y_w  \cdot v_c $
  

Combine the above 2 cases we have: $\frac {\partial J}{\partial U} =  $

#### 1.d Derivative of sigmoid function

$$\begin{align}
\sigma(x) &= \frac {1} {1 + \exp(-x)}\Rightarrow \\
\sigma(x) &= \frac {\exp(x)} {1 + \exp(x)} \Rightarrow \\
\sigma(x) &= \exp(x)\frac {1} {1 + \exp(x)} \Rightarrow \\
\sigma'(x) &= (\exp(x))'\frac {1} {1 + \exp(x)} + \exp(x)(\frac {1} {1 + \exp(x)})' \Rightarrow \\
\sigma'(x) &= \frac {\exp(x)} {1 + \exp(x)} + \exp(x) \frac {\exp(x)} {(1 + \exp(x))^2} \Rightarrow \\
\sigma'(x) &= \frac {\exp(x)} {1 + \exp(x)} (1 - \frac {\exp(x)} {1 + \exp(x)}) \Rightarrow \\
\sigma'(x) &= \sigma(x)(1 - \sigma(x))
\end{align}$$

#### 1.e Gradient w.r.t center/output word vectors when using negative sampling loss

Given: <br>
$$\hat y_o = p(o|c) = \sigma(u_o^Tv_c) = \frac{1}{1 + \exp(-u_o^Tv_c)}$$  <br>
$$J_{neg-sample}(o,v_c,U) = -\log(\sigma(u_o^T v_c)) - \sum_{k=1}^K \log(\sigma(-u_k^T v_c))$$  <br>
Find $$\frac {\partial J_{neg-sample}}{\partial v_c}$$ <br>

* Derivative w.r.t. $v_c$ will contain two terms for the expected and negative each <br>

$$\frac{\partial J}{\partial v_c} = 
\frac{\partial J}{\partial(-log(\sigma(u_o^Tv_c)))} \frac{\partial(-log(\sigma(u_o^Tv_c)))}{\partial u_o^Tv_c} \frac{\partial u_o^Tv_c}{\partial v_c} + 
\frac{\partial J}{\partial(-\sum_{k=1}^K log(\sigma(-u_k^Tv_c)))} \frac{\partial(-\sum_{k=1}^K log(\sigma(-u_k^Tv_c)))}{\partial u_o^Tv_c} \frac{\partial u_o^Tv_c}{\partial v_c}$$ <br>

$$ = -\frac{1}{\sigma(u_o^Tv_c)} \sigma(u_o^Tv_c)(1 - \sigma(u_o^Tv_c))u_o  
+\sum_{k=1}^K (\frac{1}{\sigma(-u_k^Tv_c)} \sigma(-u_k^Tv_c)(1 - \sigma(-u_k^Tv_c))u_k)$$

Derivative w.r.t. $u_o$ will contain only one term because $o \notin {1,2,...,K}$ <br> <br>
$$\frac {\partial J}{\partial u_o} = -\frac{1}{\sigma(u_o^Tv_c)} \sigma(u_o^Tv_c)(1 - \sigma(u_o^Tv_c))v_c $$
$$\frac {\partial J}{\partial u_{i \ne o}} = \frac{1}{\sigma(-u_k^Tv_c)} \sigma(-u_k^Tv_c)(1 - \sigma(-u_k^Tv_c))v_c$$ <br>
and 0 for all other output words $u_j$. Beware that since $u_k$ is randomly selected, it is possible that one word is sampled multiple times, hence the derivitive w.r.t. this word should also be accumulated multiple times.

Negative sampling is much more effective in that it needs no softmax computation, for each output word a softmax needs V vector multiplication while negative sampling only needs K+1 vector multiplication.<br>
So the speed-up ratio is up to V/K, which in a lot of cases we may have millions of vocabulary while we can choose a K of only several hundreds.